
The Schroedinger equation is:
$$i \hbar \partial_t \Psi(t,x) = H \cdot \Psi(t,x)$$

$$H = \frac{-\hbar^2}{2 \mu} (\partial_x)^2 + V(t,x)$$

We use the split step Fourier method. We borrow it from this [post](http://jakevdp.github.io/blog/2012/09/05/quantum-python/):



1. Choose $a$, $b$, $N$, and $k_0$ as above, sufficient to represent the initial state of your wave function $\psi(x)$. (Warning: this is perhaps the hardest part of the entire solution. If limits in $x$ or $k$ are chosen which do not suit your problem, then the approximations used above can destroy the accuracy of the calculation!) Once these are chosen, then $\Delta x = (b - a) / N$ and $\Delta k = 2\pi / (b - a)$. Define $x_n = a + n \Delta x$ and $k_m = k_0 + m \Delta k$.

2. Discretize the wave-functions on this grid. Let $\psi_n(t) = \psi(x_n, t)$, $V_n = V(x_n)$, and $\widetilde{\psi}_m = \widetilde{\psi}(k_m, t)$.
To progress the system by a time-step $\Delta t$, perform the following:
* Compute a half-step in $x$: $\psi_n \longleftarrow \psi_n \exp[-i (\Delta t / 2) (V_n / \hbar)]$
* Calculate $\widetilde{\psi}_m$ from $\psi_n$ using the FFT.
* Compute a full-step in $k$: $\widetilde{\psi}_m \longleftarrow \widetilde{\psi}_m \exp[-i \hbar (k \cdot k) \Delta t / (2 m)]$
*  Calculate $\psi_n$ from $\widetilde{\psi}_m$ using the inverse FFT.
* Compute a second half-step in $x$: $\psi_n \longleftarrow \psi_n \exp[-i (\Delta t / 2)(V_n / \hbar)]$
1. Goto step 3 until the desired time is reached.


In [1]:
# We use the split step Fourier method. 
# We borrow it from this [post](http://jakevdp.github.io/blog/2012/09/05/quantum-python/)
x_min = -1.
x_max = 1.
lx = x_max - x_min
N = 100
dx = lx/N
xs = linspace(x_min, x_max - dx, N)
@assert step(xs) ≈ dx
k_min = -π/dx
dk = 2π/lx
k_max = k_min + N * dk
ks = linspace(k_min, k_max - dk, N)

dt = dx^2
ħ = 1
μ = 1 # mass

V(x) = 1x^2
Vs = map(V, xs)

ψ_0 = normalize!(map(x -> exp(-100(x - 0)^2), xs))/√(lx)
ρ_0 = @. abs(ψ_0)^2
@assert sum(ρ_0) * lx ≈ 1

const eigenvalues_propagator_x = @. exp(-im*(dt/2)*Vs/ħ)
const eigenvalues_propagator_k = @. exp(-im*ħ*ks^2 * dt /(2μ))

@assert all(@. abs(eigenvalues_propagator_x) ≈ 1) "eigenvalues must be unitary."
@assert all(@. abs(eigenvalues_propagator_k) ≈ 1) "eigenvalues must be unitary."

propagate_x(ψ) = eigenvalues_propagator_x .* ψ
propagate_k(ψ) = ifft(eigenvalues_propagator_k .* fft(ψ))
propagate(ψ) = propagate_x(propagate_k(ψ))
function propagate(ψ, n)
    norm_in = norm(ψ)
    for _ in 1:n
        ψ = propagate(ψ)
    end
    norm_out = norm(ψ)
    @assert norm_in ≈ norm_out "propagation must be unitary."
    ψ
end

propagate (generic function with 2 methods)

In [3]:
using Plots
ψ = ψ_0

@gif for n in 1:100
    ψ = propagate(ψ, 1)
    ρ = @. abs(ψ)^2
    plot(xs, ρ, label="rho_$n")
    plot!(xs, Vs/10, label="Potential")
end every 1

INFO: Saved animation to /home/jan/Documents/notebooks/SplitFourierSchroedinger/tmp.gif


Plots.AnimatedGif("/home/jan/Documents/notebooks/SplitFourierSchroedinger/tmp.gif")